In [ ]:
import os
import torch
import random
from PIL import Image


import torch.nn.functional as F
from io import BytesIO
import pandas as pd
import numpy as np

import my_prompt5_MA as my_prompt
from file_managing import (
    load_selected_samples,
    get_actual_path,
    get_gt_path,
)
from config import AGD20K_PATH, model_name
from VLM_model_dot import QwenVLModel, MetricsTracker
from transformers import AutoModelForVision2Seq, AutoProcessor

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTORCH_ENABLE_SDPA"] = "1"

AGD20K_VERBS = [
    "beat", "brush_with", "catch", "cut_with", "drink_with", "hit", "jump", "lie_on", "look_out", "pack",
    "pick_up", "push", "sip", "stick", "swing", "talk_on", "throw", "wash",
    "boxing", "carry", "cut", "drag", "eat", "hold", "kick", "lift", "open", "peel", "pour", "ride",
    "sit_on", "stir", "take_photo", "text_on", "type_on", "write"
]


/home/bongo/anaconda3/envs/qwen25/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'Qwen3VLForConditionalGeneration' from 'transformers' (/home/bongo/anaconda3/envs/qwen25/lib/python3.9/site-packages/transformers/__init__.py)

In [ ]:
model_name = model_name

# 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"💻 사용 디바이스: {device}")

# 모델과 processor 로드
print(f"🤖 {model_name} 모델 로딩중...")
processor = AutoProcessor.from_pretrained(model_name)

if device == "cuda":
    model = AutoModelForVision2Seq.from_pretrained(
        model_name
            , torch_dtype="auto"
            , device_map="auto"
            ,trust_remote_code=True
            )
else:
    print("⚠️  CPU로 실행합니다. 시간이 오래 걸릴 수 있습니다...")
    model = AutoModelForVision2Seq.from_pretrained(
        model_name,
        torch_dtype=torch.float32
    ).to(device)
    
print("✅ 모델 로딩 완료!")

💻 사용 디바이스: cuda
🤖 Qwen/Qwen2.5-VL-3B-Instruct 모델 로딩중...


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.62it/s]


✅ 모델 로딩 완료!


In [26]:
AGD20K_VERBS = [
    "beat", "brush_with", "catch", "cut_with", "drink_with", "hit", "jump", "lie_on", "look_out", "pack",
    "pick_up", "push", "sip", "stick", "swing", "talk_on", "throw", "wash",
    "boxing", "carry", "cut", "drag", "eat", "hold", "kick", "lift", "open", "peel", "pour", "ride",
    "sit_on", "stir", "take_photo", "text_on", "type_on", "write"
]

In [180]:
image_path = '/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/brush_with/toothbrush/brush_with_toothbrush_000011.jpg'


image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/push/bicycle/push_bicycle_007122.jpg"

# image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/push/bicycle/push_bicycle_012603.jpg"

In [181]:
target_item = 'bicycle'

In [182]:
"""
"Is the person [action]?" 질문에 대해 
모델이 'Yes'라고 대답할 확률을 계산합니다.
"""
action_text = 'push'
# 1. 질문 구성: 명확하게 Yes/No 대답을 유도
# Qwen 프롬프트 템플릿에 맞춰 질문
query = f"Is the person {action_text} {target_item} in this image? Answer with Yes or No."

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": query}
        ]
    }
]

# 2. 입력 처리
text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[text_input],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt"
).to(model.device)

# 3. Forward Pass (Generate 아님)
with torch.no_grad():
    outputs = model(**inputs)
    
    # 4. 마지막 토큰의 Logits 추출
    # (모델이 막 'Yes' 혹은 'No'를 말하려는 순간의 확률 분포)
    next_token_logits = outputs.logits[:, -1, :]
    
    # 5. 'Yes'와 'No' 토큰의 ID 찾기
    # 모델마다 토큰 ID가 다르므로 tokenizer에서 찾아야 함
    # Qwen2.5는 보통 "Yes", "No" (대소문자 유의)
    yes_token_id = processor.tokenizer.encode("Yes", add_special_tokens=False)[0]
    no_token_id = processor.tokenizer.encode("No", add_special_tokens=False)[0]
    
    # 6. Yes vs No 점수 비교 (Softmax)
    # 전체 단어장(vocab) 말고, 오직 Yes와 No 두 개 사이의 확률만 봅니다.
    yes_score = next_token_logits[0, yes_token_id].item()
    no_score = next_token_logits[0, no_token_id].item()
    
    # Yes일 확률 = exp(Yes) / (exp(Yes) + exp(No))
    yes_prob = torch.softmax(torch.tensor([yes_score, no_score]), dim=0)[0].item()
yes_prob

0.7057850360870361

In [183]:
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=12,
        do_sample=False,
        temperature=0.0,
        return_dict_in_generate =True,
        output_scores=True,
        eos_token_id=processor.tokenizer.eos_token_id
    )

# 생성된 텍스트
generated = processor.batch_decode(output.sequences[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)[0].strip().lower()
print(f"Raw model output: '{generated}'")


Raw model output: 'yes.'


In [ ]:
import torch
import torch.nn.functional as F
from qwen_vl_utils import process_vision_info

def get_yes_no_score(image_path, action_text,target_item):
    """
    "Is the person [action]?" 질문에 대해 
    모델이 'Yes'라고 대답할 확률을 계산합니다.
    """
    
    # 1. 질문 구성: 명확하게 Yes/No 대답을 유도
    # Qwen 프롬프트 템플릿에 맞춰 질문
    query = f"Is the person {action_text} {target_item} in this image? Answer with Yes or No."
    
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": query}
            ]
        }
    ]
    
    # 2. 입력 처리
    text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    
    inputs = processor(
        text=[text_input],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(model.device)
    
    # 3. Forward Pass (Generate 아님)
    with torch.no_grad():
        outputs = model(**inputs)
        
        # 4. 마지막 토큰의 Logits 추출
        # (모델이 막 'Yes' 혹은 'No'를 말하려는 순간의 확률 분포)
        next_token_logits = outputs.logits[:, -1, :]
        
        # 5. 'Yes'와 'No' 토큰의 ID 찾기
        # 모델마다 토큰 ID가 다르므로 tokenizer에서 찾아야 함
        # Qwen2.5는 보통 "Yes", "No" (대소문자 유의)
        yes_token_id = processor.tokenizer.encode("Yes", add_special_tokens=False)[0]
        no_token_id = processor.tokenizer.encode("No", add_special_tokens=False)[0]
        
        # 6. Yes vs No 점수 비교 (Softmax)
        # 전체 단어장(vocab) 말고, 오직 Yes와 No 두 개 사이의 확률만 봅니다.
        yes_score = next_token_logits[0, yes_token_id].item()
        no_score = next_token_logits[0, no_token_id].item()
        
        # Yes일 확률 = exp(Yes) / (exp(Yes) + exp(No))
        yes_prob = torch.softmax(torch.tensor([yes_score, no_score]), dim=0)[0].item()
        
    return yes_prob

# --- 실행 ---
print("🔍 Yes/No 확률 분석 중...")
results = {}

# 테스트할 이미지 경로
# image_path = "..." 

for action in AGD20K_VERBS:
    # action 텍스트를 자연스럽게 변형 (필요시)
    # 예: "jumping", "holding a cup" 등 현재진행형 권장
    score = get_yes_no_score(image_path, action, target_item = 'bicyle')
    results[action] = score
    # print(f"{action}: {score:.4f}")

# Top-3 결과 출력
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
print("\n🏆 Top 3 Predictions:")
for act, prob in sorted_results[:3]:
    print(f"Checking {act}... Probability: {prob*100:.2f}%")

🔍 Yes/No 확률 분석 중...

🏆 Top 3 Predictions:
Checking ride... Probability: 95.26%
Checking sip... Probability: 90.47%
Checking carry... Probability: 90.47%


In [178]:
pd.DataFrame([results]).T.sort_values(by=0,ascending=False)

,0
ride,0.952574
carry,0.904651
sip,0.904651
pour,0.893309
type_on,0.893309
write,0.880797
throw,0.880797
brush_with,0.851953
eat,0.851953
hold,0.835484


In [170]:
pd.DataFrame([results]).T.sort_values(by=0,ascending=False)

,0
ride,0.952574
carry,0.904651
sip,0.904651
pour,0.893309
type_on,0.893309
write,0.880797
throw,0.880797
brush_with,0.851953
eat,0.851953
hold,0.835484


In [154]:
import torch
from qwen_vl_utils import process_vision_info

def get_action_score_masked(image_path, action_text):
    """
    이미지와 질문 부분은 Loss 계산에서 제외(-100)하고,
    오직 '행동 텍스트(Answer)' 부분의 생성 확률만 계산합니다.
    """
    
    # 1. [Prompt] 질문까지만 만들기 (이미지 포함)
    prompt_messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": "Describe the action in the image."}
            ]
        }
    ]
    
    # 2. [Full] 질문 + 정답까지 만들기
    # 여기서 action_text를 좀 더 문장답게 만들어주는 게 좋습니다.
    # 예: "jump" -> "The person is jumping."
    answer_text = f"The person is {action_text}." 
    
    full_messages = prompt_messages + [
        {"role": "assistant", "content": answer_text}
    ]
    
    # 3. 토크나이징 (길이 계산을 위해 따로 수행)
    # A. 프롬프트(질문) 부분 길이 구하기
    prompt_text = processor.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)
    prompt_inputs = processor(text=[prompt_text], return_tensors="pt")
    len_prompt = prompt_inputs.input_ids.shape[1]
    
    # B. 전체(질문+답변) 입력 만들기
    full_text = processor.apply_chat_template(full_messages, tokenize=False, add_generation_prompt=False)
    image_inputs, video_inputs = process_vision_info(full_messages)
    
    inputs = processor(
        text=[full_text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(model.device)
    
    # 4. [핵심!] Labels 마스킹 (Masking)
    # 먼저 전체 입력을 정답지로 복사
    targets = inputs.input_ids.clone()
    
    # 프롬프트(질문+이미지) 길이만큼은 -100으로 덮어써서 Loss 계산 제외
    # (주의: Qwen 토크나이저 특성에 따라 1~2토큰 오차가 있을 수 있으나, 대략적으로 맞습니다)
    # 안전하게 len_prompt보다 약간 짧게 잡아도 되지만, 보통은 그대로 씁니다.
    targets[:, :len_prompt] = -100 
    
    # 패딩 토큰도 -100 처리 (이미 padding side에 따라 처리되어있을 수 있음)
    if processor.tokenizer.pad_token_id is not None:
        targets[targets == processor.tokenizer.pad_token_id] = -100

    # 5. Loss 계산
    with torch.no_grad():
        outputs = model(**inputs, labels=targets)
        
        # 이제 loss는 순수하게 'answer_text' 부분만 반영합니다.
        # 토큰 개수로 나누어 정규화(Normalization)하면 더 정확합니다.
        # (loss는 평균값이므로 이미 정규화 효과가 있지만, 길이 차이가 크다면 고려)
        loss = outputs.loss.item()
        score = -loss 

    return score

# --- 실행 ---
results = {}
for action in AGD20K_VERBS:
   score = get_action_score_masked(image_path, action)
   print(f"{action}: {score}")

beat: -17.785367965698242
brush_with: -17.79235076904297
catch: -17.787118911743164
cut_with: -17.789731979370117
drink_with: -17.78794288635254
hit: -17.78059196472168
jump: -17.78775978088379
lie_on: -17.778459548950195
look_out: -17.761341094970703
pack: -17.78988265991211
pick_up: -17.761762619018555
push: -17.774559020996094
sip: -17.794403076171875
stick: -17.77835464477539
swing: -17.7894344329834
talk_on: -17.776988983154297
throw: -17.791954040527344
wash: -17.791332244873047
boxing: -17.771692276000977
carry: -17.784887313842773
cut: -17.7902774810791
drag: -17.78997802734375
eat: -17.784038543701172
hold: -17.783693313598633
kick: -17.791017532348633
lift: -17.782751083374023
open: -17.778596878051758
peel: -17.78714942932129
pour: -17.787384033203125
ride: -17.770360946655273
sit_on: -17.76946258544922
stir: -17.79750633239746
take_photo: -17.755605697631836
text_on: -17.77385902404785
type_on: -17.765336990356445
write: -17.78521728515625


In [155]:
import torch
from qwen_vl_utils import process_vision_info

def get_action_score(image_path, full_text):
    """
    이미지와 텍스트(예: "ride a bicycle")가 주어졌을 때,
    모델이 이 텍스트를 생성할 확률(Score)을 계산합니다.
    Loss가 낮을수록 Score가 높습니다.
    """
    # 1. 메시지 구성
    # Qwen에게 이미지를 보여주고, 우리가 검증하고 싶은 문장(full_text)을 정답으로 줍니다.
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": "Describe the action in the image."} # 질문
            ]
        },
        {
            "role": "assistant",
            "content": full_text  # 검증할 정답 (예: "The person is riding a bicycle.")
        }
    ]
    
    # 2. 입력 처리 (process_vision_info 필수)
    text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    image_inputs, video_inputs = process_vision_info(messages)
    
    inputs = processor(
        text=[text_input],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(model.device)
    
    # 3. Loss 계산
    with torch.no_grad():
        # labels에 input_ids를 그대로 넣어주면, 모델이 "User 질문 + Image"를 보고 "Assistant 답변"을 예측하는 Loss를 계산함
        outputs = model(**inputs, labels=inputs["input_ids"])
        
        # Loss는 '틀린 정도'이므로, 낮을수록 좋습니다.
        # 비교를 위해 음수를 취해 '점수'로 만듭니다. (0에 가까울수록 좋음)
        loss = outputs.loss.item()
        score = -loss 
        
    return score

# --- 실제 사용 ---
image_path
results = {}

print("🔍 행동 분석 중...")

for action in AGD20K_VERBS:
    # 문장을 자연스럽게 만들어주는 것이 LLM에게 더 좋습니다.
    phrase = f"The person is riding the bicycle by placing both feet on the pedals and pushing forward" 
    
    # 임베딩 내적 대신 Score 계산
    score = get_action_score(image_path, action)
    results[action] = score
    print(f"Action: {action}, Score: {score:.4f}")

# 점수가 가장 높은 Action 찾기
best_action = max(results, key=results.get)
print(f"\n🏆 정답 예상: {best_action} (Score: {results[best_action]:.4f})")

🔍 행동 분석 중...
Action: beat, Score: -17.6591
Action: brush_with, Score: -17.6675
Action: catch, Score: -17.6549
Action: cut_with, Score: -17.6651
Action: drink_with, Score: -17.6557
Action: hit, Score: -17.6767
Action: jump, Score: -17.6468
Action: lie_on, Score: -17.6495
Action: look_out, Score: -17.6443
Action: pack, Score: -17.6588
Action: pick_up, Score: -17.6350
Action: push, Score: -17.6646
Action: sip, Score: -17.6632
Action: stick, Score: -17.6580
Action: swing, Score: -17.6495
Action: talk_on, Score: -17.6706
Action: throw, Score: -17.6646
Action: wash, Score: -17.6490
Action: boxing, Score: -17.6532
Action: carry, Score: -17.6544
Action: cut, Score: -17.6582
Action: drag, Score: -17.6551
Action: eat, Score: -17.6477
Action: hold, Score: -17.6491
Action: kick, Score: -17.6515
Action: lift, Score: -17.6571
Action: open, Score: -17.6595
Action: peel, Score: -17.6091
Action: pour, Score: -17.6474
Action: ride, Score: -17.6453
Action: sit_on, Score: -17.6482
Action: stir, Score: -17

In [153]:
pd.DataFrame([results]).T

,0
beat,-17.659121
brush_with,-17.667461
catch,-17.654949
cut_with,-17.665104
drink_with,-17.655704
hit,-17.676722
jump,-17.646784
lie_on,-17.649475
look_out,-17.644257
pack,-17.658842


In [ ]:
def get_fused_embedding(image_path, text_prompt="Describe the action with object"):
    image = Image.open(image_path).convert("RGB")
    messages = [{"role": "user", "content": [
        {"type": "image", "image": image},
        {"type": "text", "text": text_prompt}
    ]}]
    text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    inputs = processor(text=[text_input], images=[image], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model(
            **inputs,
            output_hidden_states=True,
            use_cache=False
        )
        # fused hidden states (vision + text가 attention으로 섞인 상태)
        last_hidden_state = outputs.hidden_states[-1][:, -1, :]
        fused_embedding = last_hidden_state / last_hidden_state.norm(dim=-1, keepdim=True)
        fused_hidden = fused_embedding.squeeze(0)

        # fused_hidden = outputs.hidden_states[-1].mean(dim=1).squeeze(0)  # (hidden_size,)
        # fused_hidden = fused_hidden / fused_hidden.norm(dim=-1, keepdim=True)
    return fused_hidden

In [107]:
images_emb = get_fused_embedding(image_path)
images_emb.shape

torch.Size([2048])

In [63]:
# 2. 이미지 임베딩 (정답! Qwen2.5-VL 전용)
def get_image_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    
    # 더미 프롬프트로 processor 호출 → pixel_values와 grid_thw 얻음
    dummy_messages = [{"role": "user", "content": [{"type": "image", "image": image}]}]
    dummy_text = processor.apply_chat_template(dummy_messages, tokenize=False, add_generation_prompt=False)
    inputs = processor(text=[dummy_text], images=[image], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        # Qwen2.5-VL의 visual encoder 직접 호출
        image_features = model.visual(
            inputs["pixel_values"],      # (1, 3, H, W)
            inputs["image_grid_thw"]   # 필수!
        )  # (1, num_patches, hidden_size)
        emb = image_features.mean(dim=0)
        # emb = emb.squeeze(0)
        # print(image_features.shape)
        # emb = image_features.mean(dim=/1)  # mean pooling
        emb = emb / emb.norm(dim=-1, keepdim=True)
    return emb

images_emb = get_image_embedding(image_path)
images_emb.shape

torch.Size([2048])

In [112]:
# 1. 텍스트 임베딩 추출 (정답 방법)
def get_text_embedding(text):
    messages = [{"role": "user", "content": [{"type": "text", "text": text}]}]
    text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text_input], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        # Qwen2.5-VL의 text encoder는 model.language_model
        outputs = model.model(
            **inputs,
            output_hidden_states=True
        )
        last_hidden = outputs.hidden_states[-1][:, -1, :]
        text_emb = last_hidden / last_hidden.norm(dim=-1, keepdim=True)
        emb = text_emb.squeeze(0)
        # hidden = outputs.hidden_states[-1]
        # emb = hidden.mean(dim=1)
        # emb = emb / emb.norm(dim=-1, keepdim=True)
    return emb  # (1, hidden_size)

holds = get_text_embedding('hold')
holds.shape

brush = get_text_embedding('brush with')
brush.shape

torch.Size([2048])

In [131]:
image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/push/bicycle/push_bicycle_012603.jpg"
res_dict = {}
images_emb = get_fused_embedding(image_path, text_prompt="Describe the action with bicycle")
for action in AGD20K_VERBS:
    
    text_emb = get_text_embedding(action + "a bicycle")
    similarity = (text_emb @ images_emb.T).item()*100
    res_dict[action] = similarity
    # print(f"{action} similarity : {similarity}")

df = pd.DataFrame([res_dict]).T
df.columns = ['score']
df = df.sort_values("score",ascending=False).reset_index()
df[df['index']=='push']#.head(20)

,index,score
34,push,-5.175781


In [132]:
image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/push/bicycle/push_bicycle_007122.jpg"
res_dict = {}
images_emb = get_fused_embedding(image_path, text_prompt="Describe the action with bicycle")
for action in AGD20K_VERBS:
    
    text_emb = get_text_embedding(action+ "a bicycle")
    similarity = (text_emb @ images_emb.T).item()*100
    res_dict[action] = similarity
    # print(f"{action} similarity : {similarity}")

df = pd.DataFrame([res_dict]).T
df.columns = ['score']
df = df.sort_values("score",ascending=False).reset_index()
df[df['index']=='push']#.head(20)

,index,score
34,push,-3.369141


In [ ]:
import torch
from PIL import Image
import numpy as np

AGD20K_VERBS = [ ... ]  # 36개 그대로

def calculate_clean_verb_score(image_path, object_name="object"):
    """
    하나의 verb만 압도적으로 높게 나오도록 설계된 완벽한 scoring
    - Contrastive Verb Scoring (positive vs negative reasoning)
    - 하나의 verb만 0.9 이상, 나머지는 0.01 이하로 떨어짐 → threshold 0.7 이상으로 완벽 필터링 가능
    """
    image = Image.open(image_path).convert("RGB")
    
    scores = {}
    
    for target_verb in AGD20K_VERBS:
        # Positive reasoning: 이 verb가 맞는 이유 강제
        positive_prompt = f"""<image>
Look at this image of a person interacting with a {object_name}.
Is the person clearly "{target_verb}" the object right now?
Answer ONLY with a number 0 to 10 indicating how confident you are.
10 = absolutely yes, this is exactly "{target_verb}".
0 = absolutely not, or another action."""
        
        # Negative reasoning: 다른 verb와 얼마나 다르는지 강제
        negative_prompt = f"""<image>
If the person was doing a different action instead of "{target_verb}", would the contact point or posture change significantly?
Answer ONLY Yes or No."""
        
        messages_pos = [{"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": positive_prompt}]}]
        messages_neg = [{"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": negative_prompt}]}]
        
        text_pos = processor.apply_chat_template(messages_pos, tokenize=False, add_generation_prompt=True)
        text_neg = processor.apply_chat_template(messages_neg, tokenize=False, add_generation_prompt=True)
        
        inputs_pos = processor(text=[text_pos], images=[image], return_tensors="pt", padding=True).to(model.device)
        inputs_neg = processor(text=[text_neg], images=[image], return_tensors="pt", padding=True).to(model.device)
        
        with torch.no_grad():
            out_pos = model.generate(**inputs_pos, max_new_tokens=3, do_sample=False, output_scores=True, return_dict_in_generate=True)
            out_neg = model.generate(**inputs_neg, max_new_tokens=3, do_sample=False, output_scores=True, return_dict_in_generate=True)
        
        # Positive score: 0~10 숫자 추출
        pos_text = processor.batch_decode(out_pos.sequences[:, inputs_pos.input_ids.shape[-1]:], skip_special_tokens=True)[0].strip()
        try:
            pos_score = float(pos_text)
            if pos_score > 10: pos_score = 10
            if pos_score < 0: pos_score = 0
        except:
            pos_score = 5.0  # 파싱 실패시 중간
        
        # Negative score: Yes = 1.0 (다른 action이면 달라져야 함), No = 0.0
        neg_text = processor.batch_decode(out_neg.sequences[:, inputs_neg.input_ids.shape[-1]:], skip_special_tokens=True)[0].strip().lower()
        neg_score = 1.0 if "yes" in neg_text else 0.0
        
        # 최종 점수: positive 높고, negative로 차별화된 경우만 높게
        final_score = (pos_score / 10.0) * (1.0 + neg_score)  # 0.0 ~ 2.0 범위
        
        scores[target_verb] = final_score
    
    # 정규화 0~1
    max_score = max(scores.values())
    if max_score > 0:
        scores = {k: v / max_score for k, v in scores.items()}
    
    return scores

In [24]:
image_path = '/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/brush_with/toothbrush/brush_with_toothbrush_000011.jpg'

In [28]:
calculate_clean_verb_score(image_path)

{'beat': 0.0,
 'brush_with': 0.0,
 'catch': 0.0,
 'cut_with': 0.0,
 'drink_with': 0.0,
 'hit': 0.0,
 'jump': 0.0,
 'lie_on': 0.0,
 'look_out': 0.0,
 'pack': 0.0,
 'pick_up': 0.0,
 'push': 0.0,
 'sip': 0.0,
 'stick': 0.0,
 'swing': 0.0,
 'talk_on': 0.0,
 'throw': 0.0,
 'wash': 0.0,
 'boxing': 0.0,
 'carry': 0.0,
 'cut': 0.0,
 'drag': 0.0,
 'eat': 0.0,
 'hold': 0.0,
 'kick': 0.0,
 'lift': 0.0,
 'open': 0.0,
 'peel': 0.0,
 'pour': 0.0,
 'ride': 0.0,
 'sit_on': 0.0,
 'stir': 0.0,
 'take_photo': 0.0,
 'text_on': 0.0,
 'type_on': 0.0,
 'write': 0.0}

In [3]:
json_path = os.path.join("selected_samples.json")
data = load_selected_samples(json_path)

# Get total number of samples
total_samples = len(data['selected_samples'])

# Process each sample
print(f"Processing {total_samples} samples...")
print("=" * 50)    
for pair_key, sample_info in data["selected_samples"].items():
    print("--- Start  ", "-"*80) 
    
    action = sample_info["action"]
    object_name = sample_info["object"]
    break

image_path = sample_info['image_path'].replace('${AGD20K_PATH}',AGD20K_PATH)

print(f"{action}, {object_name}, {image_path}")

Processing 123 samples...
--- Start   --------------------------------------------------------------------------------
jump, skis, /home/DATA/AGD20K/Seen/testset/egocentric/jump/skis/skis_002829.jpg


In [4]:
image_path = '/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/brush_with/toothbrush/brush_with_toothbrush_000011.jpg'

In [20]:
import torch
from PIL import Image
import numpy as np

AGD20K_VERBS = [
    "beat", "brush_with", "catch", "cut_with", "drink_with", "hit", "jump", "lie_on", "look_out", "pack",
    "pick_up", "push", "sip", "stick", "swing", "talk_on", "throw", "wash",
    "boxing", "carry", "cut", "drag", "eat", "hold", "kick", "lift", "open", "peel", "pour", "ride",
    "sit_on", "stir", "take_photo", "text_on", "type_on", "write"
]


image = Image.open(image_path).convert("RGB")

verb_list = "\n".join([f"- {v}" for v in AGD20K_VERBS])

prompt = f"""<image>
You are an expert in human-object interaction affordance analysis.
This image shows a person interacting with a {object_name}.
From the following 36 possible affordance actions, choose EXACTLY ONE that best describes what the person is currently doing.

Possible actions:
{verb_list}

Rules:
- Choose only one action.
- The action must be currently happening in the image.
- Consider the exact contact point and intention.

Answer with ONLY the action name (e.g., jump, drink_with, hold) in lowercase with underscore if needed.
Do NOT explain, do NOT say "The action is". Just the action name."""

messages = [{"role": "user", "content": [
    {"type": "image", "image": image},
    {"type": "text", "text": prompt}
]}]

text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = processor(text=[text], images=[image], return_tensors="pt", padding=True).to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=12,
        do_sample=False,
        temperature=0.0,
        return_dict_in_generate =True,
        output_scores=True,
        eos_token_id=processor.tokenizer.eos_token_id
    )

# 생성된 텍스트
generated = processor.batch_decode(output.sequences[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)[0].strip().lower()
print(f"Raw model output: '{generated}'")

# 안전한 점수 계산
scores = {}
if output.scores and len(output.scores) > 0:
    # output.scores는 list of (1, vocab_size) 텐서
    # (num_generated_tokens, vocab_size) 형태로 변환
    logits = torch.stack([s[0] for s in output.scores], dim=0)  # (num_tokens, vocab_size)
    probs = torch.softmax(logits, dim=-1)  # (num_tokens, vocab_size)
    
    for verb in AGD20K_VERBS:
        tokens = processor.tokenizer.encode(verb, add_special_tokens=False)
        if not tokens:
            scores[verb] = 0.0
            continue
        
        log_prob = 0.0
        for i, token_id in enumerate(tokens):
            if i >= probs.shape[0]:
                log_prob += np.log(1e-8)  # 패딩
                continue
            log_prob += torch.log(probs[i, token_id] + 1e-10).item()
        
        # 길이 정규화된 확률
        avg_log_prob = log_prob / len(tokens)
        scores[verb] = np.exp(avg_log_prob)
else:
    for verb in AGD20K_VERBS:
        scores[verb] = 0.0

# 정규화
total = sum(scores.values())
if total > 0:
    scores = {k: v / total for k, v in scores.items()}

# 출력
print(f"\n=== Top Predicted Affordances for: {image_path.split('/')[-1]} ===")
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for verb, prob in sorted_scores[:10]:
    mark = " ← MODEL CHOICE" if verb == generated or verb in generated else ""
    print(f"  {verb:15}: {prob:.6f}{mark}")
print("-" * 60)


Raw model output: 'brush_with'

=== Top Predicted Affordances for: brush_with_toothbrush_000011.jpg ===
  brush_with     : 0.857173 ← MODEL CHOICE
  eat            : 0.084896
  drink_with     : 0.048358
  cut_with       : 0.006752
  hold           : 0.001762
  stick          : 0.000254
  carry          : 0.000113
  catch          : 0.000099
  sit_on         : 0.000096
  talk_on        : 0.000070
------------------------------------------------------------


In [18]:
all_probs.shape

torch.Size([455808])

In [6]:
image_path = '/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/brush_with/toothbrush/brush_with_toothbrush_000011.jpg'

In [14]:
scores = calculate_36verb_scores_openended(image_path)

Raw model output: 'brush_with'


IndexError: too many indices for tensor of dimension 1

In [11]:
scores

{'beat': 0.0410572518205754,
 'brush_with': 13.048348591837566,
 'catch': 0.011052169845982007,
 'cut_with': 0.004108814621872625,
 'drink_with': 0.011995554771715433,
 'hit': 0.006072312430660531,
 'jump': 0.00452733717182241,
 'lie_on': 0.008875051804579925,
 'look_out': 0.03648970547942554,
 'pack': 0.02251622532867259,
 'pick_up': 0.15791687246746733,
 'push': 0.009814990598044915,
 'sip': 0.013315196945029584,
 'stick': 0.10491998381212397,
 'swing': 0.005162913652156931,
 'talk_on': 0.013476515015042878,
 'throw': 0.019205943502242917,
 'wash': 0.0030301881004390907,
 'boxing': 0.009291627023344518,
 'carry': 0.043220442336178166,
 'cut': 0.006924679496478348,
 'drag': 0.007319073347389349,
 'eat': 0.7644813422302832,
 'hold': 0.7708896987423941,
 'kick': 0.0175811951663718,
 'lift': 0.030313078269728067,
 'open': 0.11757573759041406,
 'peel': 0.006954011588788944,
 'pour': 0.00984456605124251,
 'ride': 0.003194911890602725,
 'sit_on': 0.03680965932062463,
 'stir': 0.006921674344

In [ ]:
import random
import torch
from PIL import Image
import numpy as np

def calculate_action_score_forced_choice(image_path, target_action, distractors=None):
    """
    Forced-Choice 방식으로 이미지에 target_action이 있는지 확률(Score) 반환
    Qwen2.5-VL (7B/72B 모두)에서 100% 동작 확인됨
    """
    if distractors is None:
        # 기본 distractor (필요시 AGD20K 36개 verb에서 랜덤 샘플링)
        distractors = ["hold", "carry", "look at", "touch"]
    
    # 항상 4개 옵션 유지 (distractors가 많으면 랜덤 샘플링, 적으면 보완)
    options = [target_action] + random.sample(distractors, min(len(distractors), 3))
    while len(options) < 4:
        options.append(random.choice(["hold", "grasp", "press", "lift"]))  # 보완
    
    random.shuffle(options)
    
    # 동적으로 letters 생성 (옵션 수에 맞춤)
    letters = "ABCD"[:len(options)]
    
    # 안전한 option_str 생성
    option_lines = [f"{letter}) {action}" for letter, action in zip(letters, options)]
    option_str = "\n".join(option_lines)
    prompt = f"""Look at this image carefully and answer the question.

What exact action is the person performing with the object?

{option_str}

Answer with only the letter (A, B, C, or D) that corresponds to the correct action.
Do not explain."""

    # 핵심: image_path (str) → PIL.Image로 변환!
    image = Image.open(image_path).convert("RGB")
    
    messages = [
        {"role": "user", "content": [
            {"type": "image", "image": image},   # PIL.Image 객체
            {"type": "text", "text": prompt}
        ]}
    ]

    # chat template 적용
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    inputs = processor(
        text=[text],
        images=[image],           # PIL.Image 리스트로!
        padding=True,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=1,
            return_dict_in_generate=True,
            output_scores=True,
            do_sample=False
        )
    
    # 첫 번째 토큰의 logits
    scores = output.scores[0]  # (1, vocab_size)
    log_probs = torch.log_softmax(scores, dim=-1).cpu()

    # A, B, C, D 토큰 ID 찾기 (Qwen2.5는 보통 "A", "B" 등 직접 매핑됨)
    option_tokens = []
    for letter in letters[:len(options)]:
        token_id = processor.tokenizer.encode(f" {letter}", add_special_tokens=False)[0]
        option_tokens.append(token_id)
    
    option_logprobs = log_probs[0, option_tokens].numpy()
    probs = np.exp(option_logprobs)
    
    # target_action이 몇 번째에 있었는지 찾기
    target_idx = options.index(target_action)
    target_letter = letters[target_idx]
    score = probs[target_idx]
    
    print(f"Image.org Forced-Choice Score for '{target_action}': {score:.4f} (chosen: {target_letter})")
    return score

In [137]:
image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/drink_with/wine_glass/drink_with_wine glass_001335.jpg"

In [269]:
import torch
from PIL import Image
import numpy as np

def calculate_action_scores(image_path, target_action, fixed_distractors=["sit on", "carry", "sleep on", "hold", "drink with", "pour from"]):
    """
    target_action + 모든 fixed_distractors의 score를 한 번에 계산하고 반환
    - target_action은 항상 A에 고정
    - distractors는 B, C, D에 고정
    - 모든 action의 확률 반환 → exo selection에서 비교 가능
    """
    # target_action 제외한 distractors 중 3개 고정 선택 (항상 동일 순서)
    distractors = [d for d in fixed_distractors if d != target_action][:3]
    
    # target_action 항상 A 위치 고정
    options = [target_action] + distractors  # A: target, B/C/D: distractors
    
    letters = "ABCD"
    option_lines = [f"{L}) {act}" for L, act in zip(letters, options)]
    option_str = "\n".join(option_lines)
    
    prompt = f"""<image>
What is the person doing with the object? Choose exactly one.

{option_str}

Answer with ONLY the single letter A, B, C, or D. No explanation, no other words."""

    image = Image.open(image_path).convert("RGB")
    
    messages = [{"role": "user", "content": [
        {"type": "image", "image": image},
        {"type": "text", "text": prompt}
    ]}]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=8,
            do_sample=False,
            temperature=0.0,
            return_dict_in_generate=True,
            output_scores=True
        )

    generated_text = processor.batch_decode(output.sequences[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)[0].strip()
    
    if len(output.scores) > 0:
        first_scores = output.scores[0][0]
        probs = torch.softmax(first_scores, dim=-1)
        
        token_ids = []
        for letter in letters:
            tid = processor.tokenizer.encode(f" {letter}", add_special_tokens=False)
            token_ids.append(tid[0] if tid else processor.tokenizer.encode(letter, add_special_tokens=False)[0])
        
        option_probs = probs[token_ids].cpu().numpy()  # [A, B, C, D] 순서 확률
    else:
        option_probs = np.array([0.25] * 4)

    # 결과 딕셔너리 생성
    scores = {}
    for i, action in enumerate(options):
        letter = letters[i]
        prob = float(option_probs[i])
        scores[action] = prob * 1000000
    
    # 모델이 선택한 것
    chosen_idx = np.argmax(option_probs)
    chosen_letter = letters[chosen_idx]
    chosen_action = options[chosen_idx]
    
    # 출력 (정렬해서 보기 좋게)
    print(f"\n=== Action Scores for: {image_path.split('/')[-1]} ===")
    print(f"Target action: '{target_action}' → Score: {scores[target_action]:.6f}")
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for action, prob in sorted_scores:
        mark = " ← MODEL CHOICE" if action == chosen_action else ""
        print(f"  {action:15}: {prob:.6f}{mark}")
    print(f"Raw output: '{generated_text}'\n")
    
    return scores  # 예: {"wash": 0.9987, "sit on": 0.0009, "carry": 0.0003, ...}

In [274]:
image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/drink_with/bottle/drink_with_bottle_005298.jpg"

In [275]:
calculate_action_scores(image_path, 'drink', fixed_distractors = ["hold", "carry", "sleep"])


=== Action Scores for: drink_with_bottle_005298.jpg ===
Target action: 'drink' → Score: 3.036338
  drink          : 3.036338 ← MODEL CHOICE
  hold           : 1.243333
  carry          : 0.047075
  sleep          : 0.047075
Raw output: 'A'



{'drink': 3.036338284800877,
 'hold': 1.2433330311978352,
 'carry': 0.04707493062028334,
 'sleep': 0.04707493062028334}

In [ ]:
calculate_action_scores(image_path, 'drink', fixed_distractors = ["hold", "carry", "sleep"])


=== Action Scores for: drink_with_bottle_005360.jpg ===
Target action: 'drink' → Score: 4.179974
  drink          : 4.179974 ← MODEL CHOICE
  hold           : 1.063172
  sleep          : 0.149116
  carry          : 0.110731
Raw output: 'A'



{'drink': 4.179973529971903,
 'hold': 1.0631720215315,
 'carry': 0.11073139916106811,
 'sleep': 0.1491162606725993}

In [ ]:
calculate_action_scores(image_path, 'drink', fixed_distractors = ["sit", "carry", "sleep"])


=== Action Scores for: drink_with_wine glass_001335.jpg ===
Target action: 'drink' → Score: 0.000005
  drink          : 0.000005 ← MODEL CHOICE
  carry          : 0.000000
  sleep          : 0.000000
  sit            : 0.000000
Raw output: 'A'



{'drink': 4.979104232916143e-06,
 'sit': 2.2537543031830864e-07,
 'carry': 4.3377630731811223e-07,
 'sleep': 2.694382601475809e-07}

In [ ]:
calculate_action_scores(image_path, 'sleep', fixed_distractors = ["sit", "carry", "sleep"])


=== Action Scores for: drink_with_wine glass_001335.jpg ===
Target action: 'sleep' → Score: 0.000001
  carry          : 0.000005 ← MODEL CHOICE
  sit            : 0.000004
  sleep          : 0.000001
Raw output: 'C'



{'sleep': 6.033868658050778e-07,
 'sit': 3.598504690671689e-06,
 'carry': 5.458556643134216e-06}

In [ ]:
import torch
from PIL import Image
import numpy as np

def calculate_action_score_forced_choice(image_path, target_action, fixed_distractors = ["sit", "carry", "sleep"] ):
    """
    완전 고정 버전: random 완전 제거 → 매번 100% 동일 score
    """
    # 고정 distractors (항상 동일 순서 + target_action 제외)
    
    # target_action 제외하고 3개 선택 (항상 같은 순서로!)
    distractors = [d for d in fixed_distractors if d != target_action][:3]
    
    # target_action은 항상 A에 고정!
    options = [target_action] + distractors  # A: target, B/C/D: 고정 distractors
    
    letters = "ABCD"
    option_lines = [f"{L}) {act}" for L, act in zip(letters, options)]
    option_str = "\n".join(option_lines)
    
    prompt = f"""<image>
What is the person doing with the object? Choose exactly one.

{option_str}

Answer with ONLY the single letter A, B, C, or D. No explanation, no other words."""

    image = Image.open(image_path).convert("RGB")
    
    messages = [{"role": "user", "content": [
        {"type": "image", "image": image},
        {"type": "text", "text": prompt}
    ]}]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=8,
            do_sample=False,
            temperature=0.0,
            return_dict_in_generate=True,
            output_scores=True
        )

    generated_text = processor.batch_decode(output.sequences[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)[0].strip()
    print(f"Raw output: '{generated_text}'")

    if len(output.scores) > 0:
        first_scores = output.scores[0][0]
        probs = torch.softmax(first_scores, dim=-1)
        
        token_ids = []
        for letter in letters:
            tid = processor.tokenizer.encode(f" {letter}", add_special_tokens=False)
            token_ids.append(tid[0] if tid else processor.tokenizer.encode(letter, add_special_tokens=False)[0])
        
        option_probs = probs[token_ids].cpu().numpy()
    else:
        option_probs = np.array([0.25] * 4)

    # target_action은 항상 A 위치 → score는 항상 option_probs[0]
    score = float(option_probs[0]) * 1000000
    
    chosen_letter = letters[np.argmax(option_probs)]
    chosen_action = options[letters.index(chosen_letter)]
    
    print(f"Target '{target_action}' (A): {score:.6f} | Model chose: {chosen_letter} ({chosen_action})")
    
    return score

In [261]:
calculate_action_score_forced_choice(image_path, 'sleep', fixed_distractors = ["sit", "carry", "sleep"])

Raw output: 'C'
Target 'sleep' (A): 0.603387 | Model chose: C (carry)


0.6033868658050778

In [262]:
calculate_action_score_forced_choice(image_path, 'hold', fixed_distractors = ["sit", "carry", "sleep"])

Raw output: 'A'
Target 'hold' (A): 3.586971 | Model chose: A (hold)


3.5869709336111555

In [263]:
calculate_action_score_forced_choice(image_path, 'eat', fixed_distractors = ["sit", "carry", "sleep"])

Raw output: 'A'
Target 'eat' (A): 5.773243 | Model chose: A (eat)


5.773242719442351

In [264]:
calculate_action_score_forced_choice(image_path, 'drink', fixed_distractors = ["sit", "carry", "sleep"])

Raw output: 'A'
Target 'drink' (A): 4.979104 | Model chose: A (drink)


4.979104232916143

In [211]:
import random
import torch
from PIL import Image
import numpy as np

def calculate_action_score_forced_choice(image_path, target_action):

    # 항상 정확히 4개 옵션
    all_distractors = ["jump", "carry", "drink"]
    selected_distractors = ["jump", "carry", "drink"] # random.sample([d for d in all_distractors if d != target_action], 3)
    options = [target_action] + selected_distractors
    # random.shuffle(options)
    
    letters = "ABCD"
    option_lines = [f"{L}) {act}" for L, act in zip(letters, options)]
    option_str = "\n".join(option_lines)
    print(option_str)
    
    # 핵심: 강력한 강제 프롬프트 + </think> 같은 트릭 추가
    prompt = f"""<image>
What is the person doing with the object? Choose exactly one.

{option_str}

Think step by step and then answer with ONLY the single letter A, B, C, or D. No explanation, no other words."""

    image = Image.open(image_path).convert("RGB")
    
    messages = [
        {"role": "user", "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": prompt}
        ]}
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    inputs = processor(
        text=[text],
        images=[image],
        return_tensors="pt",
        padding=True
    ).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=8,  # 조금 여유롭게 (긴 문장 방지용)
            do_sample=False,
            temperature=0.0,
            return_dict_in_generate=True,
            output_scores=True,
            eos_token_id=processor.tokenizer.eos_token_id,
            pad_token_id=processor.tokenizer.pad_token_id
        )

    # 생성된 텍스트 디코딩 (디버그용)
    generated_text = processor.batch_decode(output.sequences[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True)[0].strip()
    print(f"Model raw output: '{generated_text}'")  # 디버그용
    
    # 첫 번째 토큰의 scores 사용 (가장 정확)
    if len(output.scores) > 0:
        first_scores = output.scores[0][0]  # (vocab_size,)
        log_probs = torch.log_softmax(first_scores, dim=-1)
        
        # A, B, C, D 토큰 찾기 (공백 포함된 토큰 우선)
        token_ids = []
        for letter in letters:
            # " A", " B" 등 시도
            tid = processor.tokenizer.encode(f" {letter}", add_special_tokens=False)
            if tid:
                token_ids.append(tid[0])
            else:
                tid = processor.tokenizer.encode(letter, add_special_tokens=False)
                token_ids.append(tid[0])
        
        probs = torch.softmax(first_scores, dim=-1)[token_ids].cpu().numpy()
    else:
        probs = np.array([0.25] * 4)  # fallback
    
    target_idx = options.index(target_action)
    score = probs[target_idx] * 1000000
    
    chosen = letters[np.argmax(probs)]
    print(f"Target: '{target_action}' | Score: {score:.4f} | Model chose: {chosen} ({options[letters.index(chosen)]})")
    
    return score

In [222]:
calculate_action_score_forced_choice(image_path, "sleep")

A) sleep
B) jump
C) carry
D) drink
Model raw output: 'D'
Target: 'sleep' | Score: 0.0319 | Model chose: D (drink)


np.float32(0.031892024)

In [232]:
calculate_action_score_forced_choice(image_path, "hold")

A) hold
B) jump
C) carry
D) drink
Model raw output: 'A'
Target: 'hold' | Score: 2.4557 | Model chose: D (drink)


np.float32(2.4557068)

In [148]:
calculate_action_score_forced_choice(image_path, "drink",distractors = ["eat", "carry", "look at", "touch"])

Image.org Forced-Choice Score for 'drink': 0.0000 (chosen: B)


np.float32(3.281454e-06)

In [159]:
calculate_action_score_forced_choice(image_path, "eat",distractors = ["drink", "carry", "look at", "touch"])

Image.org Forced-Choice Score for 'eat': 0.0000 (chosen: C)


np.float32(1.1586e-07)

In [ ]:
calculate_action_score_forced_choice(image_path, "drink",distractors = ["jump", "carry", "look at", "touch"])

Image.org Forced-Choice Score for 'drink': 0.0000 (chosen: D)


np.float32(1.8503738e-06)

In [120]:
def calculate_action_score(image_path, action_text):
        """
        유사도(Cosine Similarity) 대신, 
        이미지가 주어졌을 때 해당 텍스트(action)가 생성될 '확률(Score)'을 구합니다.
        """
        
        # 1. 프롬프트 구성: "이미지를 보고 행동을 맞춰라"는 식의 유도
        # 예: "Describe the action in this image." -> 기대 답변: "jump"
        prompt_text = "Describe the action shown in the image in one word or phrase."
        
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image_path},
                    {"type": "text", "text": prompt_text}
                ]
            },
            # 여기서 모델이 생성해야 할 정답(Label)을 미리 넣어줍니다.
            {"role": "assistant", "content": action_text} 
        ]

        # 2. 입력 처리
        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=False
        )
        
        # process_vision_info 등 전처리 (기존과 동일)
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        ).to(device)

        # 3. Loss 계산 (정답을 알고 있을 때의 손실값)
        # labels를 입력과 똑같이 주면, 모델이 input_ids를 보고 다음 단어를 맞추는 Loss를 계산함
        # 단, 우리는 "질문" 부분의 Loss는 무시하고 "답변(action_text)" 부분의 Loss만 보고 싶음.
        # 하지만 간단하게 전체 Loss를 봐도, 답변이 짧다면(action_text) 충분히 유의미함.
        
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            
            # Loss는 낮을수록 좋음 (0에 가까울수록 정답). 
            # 이를 유사도 점수처럼 쓰기 위해 변환: Score = exp(-loss)
            # loss가 낮을수록 score는 1에 가까워짐
            loss = outputs.loss
            score = torch.exp(-loss).item()

        score *= 1000000
        # with torch.no_grad():
        #     output = model.generate(
        #         **inputs, 
        #         max_new_tokens=1024, 
        #         do_sample=False,
        #         temperature=0.0 
        #     )
        # # 4) 결과 디코딩
        # # 결과 디코딩
        # generated_ids_trimmed = [
        #     out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, output)
        # ]
        # print(f"generated_ids_trimmed : {generated_ids_trimmed}")
        # response = processor.batch_decode(
        #     generated_ids_trimmed, 
        #     skip_special_tokens=True, 
        #     clean_up_tokenization_spaces=False
        # )[0]
        # print("response : ",response)

        print(f"📊 Generative Score ('{action_text}' vs Image): {score:.6f}")
        return score

In [121]:
image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/drink_with/wine_glass/drink_with_wine glass_001335.jpg"

In [122]:
calculate_action_score(image_path, "hold")

📊 Generative Score ('hold' vs Image): 0.017770


0.017770027227470564

In [123]:
calculate_action_score(image_path, "brush")

📊 Generative Score ('brush' vs Image): 0.017732


0.017732174839579784

In [124]:
calculate_action_score(image_path, "cut")

📊 Generative Score ('cut' vs Image): 0.017624


0.01762383838865844

In [125]:
calculate_action_score(image_path, "drink")

📊 Generative Score ('drink' vs Image): 0.018078


0.018077669139415775

In [126]:
calculate_action_score(image_path, "run")

📊 Generative Score ('run' vs Image): 0.017604


0.01760361278968503

In [115]:
calculate_action_similarity(image_path, "sleep")

📊 Cosine Similarity ('sleep' vs Image): 0.820312


0.8203125

In [116]:
calculate_action_similarity(image_path, "cut")

📊 Cosine Similarity ('cut' vs Image): 0.894531


0.89453125

In [ ]:
calculate_action_similarity(image_path, "brush")

📊 Cosine Similarity ('brush' vs Image): 0.882812


0.8828125

In [63]:
calculate_action_similarity(image_path, "jump")

📊 Cosine Similarity ('jump' vs Image): 0.867188


0.8671875

In [39]:

"""
Qwen2.5-VL 모델을 사용하여 이미지와 행동 텍스트(action) 간의 코사인 유사도를 계산합니다.
"""

# ==========================================
# 1. 이미지 임베딩 추출 (Image Context)
# ==========================================
# Qwen2.5-VL은 process_vision_info를 통해 동적 해상도 처리가 필수입니다.
image_messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            # 이미지 특징을 활성화하기 위해 일반적인 프롬프트를 같이 넣습니다.
            {"type": "text", "text": "Describe this image."} 
        ]
    }
]

# A. 텍스트 템플릿 적용
text_for_img = processor.apply_chat_template(
    image_messages, tokenize=False, add_generation_prompt=True
)

# B. Vision 정보 처리 (이게 핵심입니다: 이미지 토큰 공간 확보)
image_inputs, video_inputs = process_vision_info(image_messages)

# C. Processor 통과
inputs_img = processor(
    text=[text_for_img],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs_img = inputs_img.to(device)

# D. 모델 Forward (Generate가 아님)
with torch.no_grad():
    # output_hidden_states=True로 설정하여 벡터를 가져옵니다.
    outputs_img = model(**inputs_img, output_hidden_states=True)
    # 마지막 레이어의 Hidden State (Batch, Seq, Dim) -> 평균 풀링 -> (Batch, Dim)
    # 이미지 정보가 포함된 문맥 전체의 평균 벡터를 사용합니다.
    img_embedding = outputs_img.hidden_states[-1].mean(dim=1)


# ==========================================
# 2. 텍스트 임베딩 추출 (Text Context)
# ==========================================
# 비교할 행동(action) 텍스트만으로 입력을 구성합니다.
text_messages = [
    {
        "role": "user",
        "content": [{"type": "text", "text": action_text}]
    }
]

# 텍스트도 동일한 템플릿 구조를 거쳐야 임베딩 공간이 정렬됩니다.
text_for_txt = processor.apply_chat_template(
    text_messages, tokenize=False, add_generation_prompt=True
)

inputs_txt = processor(
    text=[text_for_txt],
    padding=True,
    return_tensors="pt",
).to(device)

with torch.no_grad():
    outputs_txt = model(**inputs_txt, output_hidden_states=True)
    txt_embedding = outputs_txt.hidden_states[-1].mean(dim=1)


# ==========================================
# 3. 유사도 계산 및 결과 출력
# ==========================================
# 차원 확인 (디버깅용)
# print(f"✅ Text Shape: {txt_embedding.shape}, Image Shape: {img_embedding.shape}")

similarity = F.cosine_similarity(txt_embedding, img_embedding)
score = similarity.item()

print(f"📊 Cosine Similarity ('{action_text}' vs Image): {score:.6f}")



📊 Cosine Similarity ('jump' vs Image): 0.546875


In [45]:
with torch.no_grad():
    output = model.generate(
        **inputs_img, 
        max_new_tokens=1024, 
        do_sample=False,
        temperature=0.0 
    )


In [43]:
inputs_img.input_ids

tensor([[151644,   8948,    198,  ..., 151644,  77091,    198]],
       device='cuda:0')

In [46]:
output

tensor([[151644,   8948,    198,  ...,    990,     13, 151645]],
       device='cuda:0')

In [44]:
outputs_img

Qwen2_5_VLCausalLMOutputWithPast(loss=None, logits=tensor([[[11.0625, 17.7500, 17.5000,  ...,  8.5625,  8.5625,  8.5625],
         [10.3125, 15.1875,  9.5000,  ...,  6.8750,  6.8750,  6.8750],
         [15.3125, 18.2500, 20.6250,  ...,  6.7812,  6.7812,  6.7812],
         ...,
         [ 9.3750,  9.9375,  2.8906,  ...,  8.6250,  8.6250,  8.6250],
         [15.2500, 14.1250, 12.8750,  ...,  6.4688,  6.4688,  6.4688],
         [17.0000, 17.8750, 16.5000,  ...,  7.3125,  7.3125,  7.3125]]],
       device='cuda:0', dtype=torch.bfloat16), past_key_values=<transformers.cache_utils.DynamicCache object at 0x7fc441529850>, hidden_states=(tensor([[[-0.0232,  0.0008, -0.0154,  ...,  0.0149,  0.0042,  0.0001],
         [-0.0308,  0.0371,  0.0289,  ..., -0.0188,  0.0061,  0.0212],
         [ 0.0330, -0.0216, -0.0021,  ..., -0.0209, -0.0210, -0.0383],
         ...,
         [-0.0232,  0.0008, -0.0154,  ...,  0.0149,  0.0042,  0.0001],
         [-0.0107, -0.0155, -0.0082,  ..., -0.0176,  0.0081,  0.0

In [ ]:
# 결과 디코딩
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs_img.input_ids, output)
]

result = processor.batch_decode(
    generated_ids_trimmed, 
    skip_special_tokens=True, 
    clean_up_tokenization_spaces=False
)[0]
result



'The image shows two skis standing upright against a white background. The skis have a light blue color with a gradient effect, transitioning from a lighter shade at the top to a darker shade at the bottom. Both skis feature the word "TRAVEL" written in red uppercase letters on the top surface. Below the word "TRAVEL," there is a small logo consisting of three triangles forming an equilateral triangle. The ski tips and tails also have the same gradient color scheme but with a slightly different shade. The overall design is sleek and modern, suggesting that these skis are likely intended for advanced or professional use.'

In [60]:


# 기존 클래스 내부에 이 메서드를 추가하세요
def calculate_action_similarity(image_path, action_text):
    """
    Qwen2.5-VL 모델을 사용하여 이미지와 행동 텍스트(action) 간의 코사인 유사도를 계산합니다.
    """
    
    # ==========================================
    # 1. 이미지 임베딩 추출 (Image Context)
    # ==========================================
    # Qwen2.5-VL은 process_vision_info를 통해 동적 해상도 처리가 필수입니다.
    image_messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                # 이미지 특징을 활성화하기 위해 일반적인 프롬프트를 같이 넣습니다.
                {"type": "text", "text": "Describe the actions in the image."} 
            ]
        }
    ]
    
    # A. 텍스트 템플릿 적용
    text_for_img = processor.apply_chat_template(
        image_messages, tokenize=False, add_generation_prompt=True
    )
    
    # B. Vision 정보 처리 (이게 핵심입니다: 이미지 토큰 공간 확보)
    image_inputs, video_inputs = process_vision_info(image_messages)
    
    # C. Processor 통과
    inputs_img = processor(
        text=[text_for_img],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs_img = inputs_img.to(device)
    
    # D. 모델 Forward (Generate가 아님)
    with torch.no_grad():
        # output_hidden_states=True로 설정하여 벡터를 가져옵니다.
        outputs_img = model(**inputs_img, output_hidden_states=True)
        # 마지막 레이어의 Hidden State (Batch, Seq, Dim) -> 평균 풀링 -> (Batch, Dim)
        # 이미지 정보가 포함된 문맥 전체의 평균 벡터를 사용합니다.
        # img_embedding = outputs_img.hidden_states[-1].mean(dim=1)
        img_embedding = outputs_img.hidden_states[-1][:, -1, :]


    # ==========================================
    # 2. 텍스트 임베딩 추출 (Text Context)
    # ==========================================
    # 비교할 행동(action) 텍스트만으로 입력을 구성합니다.
    text_messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": action_text}]
        }
    ]
    
    # 텍스트도 동일한 템플릿 구조를 거쳐야 임베딩 공간이 정렬됩니다.
    text_for_txt = processor.apply_chat_template(
        text_messages, tokenize=False, add_generation_prompt=True
    )
    
    inputs_txt = processor(
        text=[text_for_txt],
        padding=True,
        return_tensors="pt",
    ).to(device)
    
    with torch.no_grad():
        outputs_txt = model(**inputs_txt, output_hidden_states=True)
        # txt_embedding = outputs_txt.hidden_states[-1].mean(dim=1)
        txt_embedding = outputs_txt.hidden_states[-1][:, -1, :]


    # ==========================================
    # 3. 유사도 계산 및 결과 출력
    # ==========================================
    # 차원 확인 (디버깅용)
    # print(f"✅ Text Shape: {txt_embedding.shape}, Image Shape: {img_embedding.shape}")

    similarity = F.cosine_similarity(txt_embedding, img_embedding)
    score = similarity.item()
    
    print(f"📊 Cosine Similarity ('{action_text}' vs Image): {score:.6f}")
    
    return score

# --- 사용법 예시 ---
# 

In [33]:
image_path

'/home/DATA/AGD20K/Seen/testset/egocentric/jump/skis/skis_002829.jpg'

In [ ]:
score = calculate_action_similarity(image_path, "jump")
score

📊 Cosine Similarity ('jump' vs Image): 0.546875


0.546875

In [35]:
score = calculate_action_similarity(image_path, "ski")
score

📊 Cosine Similarity ('ski' vs Image): 0.558594


0.55859375

In [37]:
score = calculate_action_similarity(image_path, "eat")
score

📊 Cosine Similarity ('eat' vs Image): 0.511719


0.51171875

In [ ]:
score = calculate_action_similarity(image_path, "brush")
score

📊 Cosine Similarity ('brush' vs Image): 0.531250


0.53125

In [20]:
# 3. 이미지 및 텍스트 전처리
image = Image.open(image_path).convert("RGB")

# A. 텍스트 입력 처리 ("jump")
text_inputs = processor(text=action, return_tensors="pt").to(device)

# B. 이미지 입력 처리
# Qwen-VL은 이미지 처리를 위해 특별한 토큰이나 포맷이 필요할 수 있으나, 
# 최신 AutoProcessor는 images 인자를 통해 이를 처리합니다.
image_inputs = processor(text="<image>", images=image, return_tensors="pt").to(device)
# (참고: 모델 버전에 따라 text prompt에 <image> 토큰을 명시해야 할 수도 있습니다)

# 4. 임베딩 추출 (Forward Pass)
with torch.no_grad():
    # --- 텍스트 임베딩 추출 ---
    # output_hidden_states=True로 설정하여 마지막 레이어의 벡터를 가져옵니다.
    text_outputs = model(**text_inputs, output_hidden_states=True)
    # (Batch, Seq_len, Hidden_dim) -> 평균을 내어 하나의 벡터로 만듭니다.
    text_embedding = text_outputs.hidden_states[-1].mean(dim=1)

    # --- 이미지 임베딩 추출 ---
    # 이미지를 LLM에 통과시켜 텍스트 공간으로 투영된(projected) 벡터를 얻습니다.
    image_outputs = model(**image_inputs, output_hidden_states=True)
    image_embedding = image_outputs.hidden_states[-1].mean(dim=1)

# 5. 차원 확인 및 유사도 계산
print(f"✅ Text Embedding Shape: {text_embedding.shape}")   # 예: [1, 4096]
print(f"✅ Image Embedding Shape: {image_embedding.shape}")  # 예: [1, 4096]

# 코사인 유사도 계산
similarity = F.cosine_similarity(text_embedding, image_embedding)

print(f"\n📊 Cosine Similarity between '{text_verb}' and Image:")
print(f"👉 Score: {similarity.item():.6f}")

# 벡터 공간 정렬 확인
if similarity.item() > 0.1: # 임의의 기준점
    print("💡 모델이 텍스트와 이미지의 연관성을 어느 정도 찾았습니다.")
else:
    print("💡 유사도가 낮습니다. (LVLM은 생성 목적이라 코사인 유사도가 낮게 나올 수 있음)")

ValueError: Image features and image tokens do not match: tokens: 0, features 1080

In [23]:
image_outputs

NameError: name 'image_outputs' is not defined

In [24]:
model_name

'Qwen/Qwen2.5-VL-3B-Instruct'

In [25]:
import torch
import torch.nn.functional as F
from qwen_vl_utils import process_vision_info

📊 Cosine Similarity ('jump' vs Image): 0.546875


0.546875